# LDA Model Training

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

## Data

In [55]:
data_file_path='DATA.csv'
data=pd.read_csv(data_file_path)
data

/home/vanka/anaconda2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,PeakID,motif_string
0,seq_40800,"motif_149,motif_162,motif_166,motif_166,motif_..."
1,seq_10314,motif_144
2,seq_18866,"motif_104,motif_126,motif_126,motif_147,motif_..."
3,seq_45348,"motif_120,motif_121,motif_128,motif_128,motif_..."
4,seq_2616,"motif_100,motif_103,motif_126,motif_126,motif_..."
...,...,...
60095,seq_5507,"motif_137,motif_137,motif_13,motif_174,motif_1..."
60096,seq_11070,"motif_132,motif_132,motif_132,motif_142,motif_..."
60097,seq_17945,"motif_161,motif_162,motif_162,motif_174,motif_..."
60098,seq_29137,"motif_104,motif_105,motif_117,motif_117,motif_..."


In [56]:
print(data[data['motif_string'].isna()])
data.dropna(subset=['motif_string'],inplace=True)
data['motif_list']=data['motif_string'].apply(lambda x:x.split(','))
data

/home/vanka/anaconda2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


          PeakID motif_string
18840  seq_57232          NaN
23892  seq_20216          NaN
29667  seq_31231          NaN
46832   seq_8465          NaN
47219  seq_46451          NaN


,PeakID,motif_string,motif_list
0,seq_40800,"motif_149,motif_162,motif_166,motif_166,motif_...","[motif_149, motif_162, motif_166, motif_166, m..."
1,seq_10314,motif_144,[motif_144]
2,seq_18866,"motif_104,motif_126,motif_126,motif_147,motif_...","[motif_104, motif_126, motif_126, motif_147, m..."
3,seq_45348,"motif_120,motif_121,motif_128,motif_128,motif_...","[motif_120, motif_121, motif_128, motif_128, m..."
4,seq_2616,"motif_100,motif_103,motif_126,motif_126,motif_...","[motif_100, motif_103, motif_126, motif_126, m..."
...,...,...,...
60095,seq_5507,"motif_137,motif_137,motif_13,motif_174,motif_1...","[motif_137, motif_137, motif_13, motif_174, mo..."
60096,seq_11070,"motif_132,motif_132,motif_132,motif_142,motif_...","[motif_132, motif_132, motif_132, motif_142, m..."
60097,seq_17945,"motif_161,motif_162,motif_162,motif_174,motif_...","[motif_161, motif_162, motif_162, motif_174, m..."
60098,seq_29137,"motif_104,motif_105,motif_117,motif_117,motif_...","[motif_104, motif_105, motif_117, motif_117, m..."


In [4]:
from gensim.corpora import Dictionary

docs=data['motif_list'].values
dictionary=Dictionary(docs)

print('Motif-Index to Motif-Name Mapping:')
for i,v in dictionary.items():
    print(f'{i} - {v}')
    if i==10:
        break
    

Motif-Index to Motif-Name Mapping:
0 - motif_149
1 - motif_162
2 - motif_166
3 - motif_174
4 - motif_199
5 - motif_230
6 - motif_248
7 - motif_295
8 - motif_60
9 - motif_66
10 - motif_98


In [5]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('BOW (Sequence-0):')
docs[0],corpus[0]

BOW (Sequence-0):


(['motif_149',
  'motif_162',
  'motif_166',
  'motif_166',
  'motif_174',
  'motif_199',
  'motif_230',
  'motif_248',
  'motif_295',
  'motif_295',
  'motif_60',
  'motif_60',
  'motif_66',
  'motif_98'],
 [(0, 1),
  (1, 1),
  (2, 2),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 2),
  (9, 1),
  (10, 1)])

## Training

In [6]:
# #setup logging for trainging metrics 
# import logging
# logging.basicConfig(filename='test_output/model_callbacks.log', filemode='w',
#                     format="%(asctime)s:%(levelname)s:%(message)s",
#                     level=logging.NOTSET)

# from gensim.models.callbacks import Callback,PerplexityMetric, ConvergenceMetric, CoherenceMetric
# perplexity_logger = PerplexityMetric(corpus=corpus, logger='shell')
# convergence_logger = ConvergenceMetric(logger='shell')
# # coherence_cv_logger = CoherenceMetric(corpus=corpus, coherence = 'c_v', texts = docs)

# %%time
# from gensim.models import LdaModel,LdaMulticore

# #HYPERPARAMETERS
# #passes = epochs
# temp = dictionary[0]
# id2word = dictionary.id2token
# lda = LdaModel(corpus, id2word=id2word, alpha='auto',eval_every = 1,\
#                eta='auto',num_topics=3, iterations=5, passes = 10,
#               callbacks=[perplexity_logger,convergence_logger])

# lda.print_topics()

# %%time
# from gensim.models import LdaModel,LdaMulticore

# id2word = dictionary.id2token
# lda = LdaMulticore(corpus, id2word=id2word,eval_every = 1,\
#                eta='auto',num_topics=3, iterations=500, passes = 100)

# import pyLDAvis.gensim

# pyLDAvis.enable_notebook()
# pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [7]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

In [9]:
%%time
from gensim.models import LdaModel,LdaMulticore

temp = dictionary[0]
id2word = dictionary.id2token
lda = LdaModel(corpus, id2word=id2word, alpha='auto',chunksize=10000,
               eta='auto',num_topics=3, iterations=1000, passes = 5)

2021-02-20 07:55:51,399 : INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
2021-02-20 07:55:51,400 : INFO : using serial LDA version on this node
2021-02-20 07:55:51,402 : INFO : running online (multi-pass) LDA training, 3 topics, 5 passes over the supplied corpus of 60095 documents, updating model once every 10000 documents, evaluating perplexity every 60095 documents, iterating 1000x with a convergence threshold of 0.001000
2021-02-20 07:55:51,404 : INFO : PROGRESS: pass 0, at document #10000/60095
2021-02-20 07:55:51,405 : DEBUG : performing inference on a chunk of 10000 documents
2021-02-20 07:56:12,221 : DEBUG : 9900/10000 documents converged within 1000 iterations
2021-02-20 07:56:12,256 : INFO : optimized alpha [0.009715915, 0.12713008, 0.22264165]
2021-02-20 07:56:12,257 : DEBUG : updating topics
2021-02-20 07:56:12,257 : INFO : merging changes from 10000 documents into a model of 60095 documents
2021-02-20 07:56:12,258 : INFO : topic #0 (0.010):

2021-02-20 07:56:34,520 : DEBUG : bound: at document #0
2021-02-20 07:56:34,561 : INFO : -5.071 per-word bound, 33.6 perplexity estimate based on a held-out corpus of 95 documents with 2924 words
2021-02-20 07:56:34,561 : INFO : PROGRESS: pass 0, at document #60095/60095
2021-02-20 07:56:34,562 : DEBUG : performing inference on a chunk of 95 documents
2021-02-20 07:56:34,591 : DEBUG : 95/95 documents converged within 1000 iterations
2021-02-20 07:56:34,592 : INFO : optimized alpha [0.024824966, 0.040896386, 0.049384993]
2021-02-20 07:56:34,593 : DEBUG : updating topics
2021-02-20 07:56:34,593 : INFO : merging changes from 95 documents into a model of 60095 documents
2021-02-20 07:56:34,595 : INFO : topic #0 (0.025): 0.034*"motif_60" + 0.021*"motif_62" + 0.019*"motif_126" + 0.018*"motif_84" + 0.016*"motif_2" + 0.016*"motif_17" + 0.015*"motif_174" + 0.015*"motif_1" + 0.015*"motif_31" + 0.014*"motif_26"
2021-02-20 07:56:34,595 : INFO : topic #1 (0.041): 0.069*"motif_60" + 0.063*"motif_184

2021-02-20 07:56:52,863 : DEBUG : updating topics
2021-02-20 07:56:52,864 : INFO : merging changes from 10000 documents into a model of 60095 documents
2021-02-20 07:56:52,865 : INFO : topic #0 (0.049): 0.035*"motif_60" + 0.024*"motif_126" + 0.022*"motif_84" + 0.022*"motif_62" + 0.018*"motif_17" + 0.016*"motif_31" + 0.016*"motif_26" + 0.015*"motif_162" + 0.014*"motif_174" + 0.014*"motif_35"
2021-02-20 07:56:52,865 : INFO : topic #1 (0.044): 0.074*"motif_60" + 0.067*"motif_184" + 0.052*"motif_218" + 0.036*"motif_166" + 0.028*"motif_98" + 0.027*"motif_284" + 0.026*"motif_171" + 0.026*"motif_94" + 0.024*"motif_181" + 0.020*"motif_158"
2021-02-20 07:56:52,866 : INFO : topic #2 (0.058): 0.039*"motif_37" + 0.036*"motif_23" + 0.034*"motif_42" + 0.029*"motif_40" + 0.029*"motif_13" + 0.026*"motif_19" + 0.022*"motif_39" + 0.018*"motif_54" + 0.017*"motif_11" + 0.017*"motif_12"
2021-02-20 07:56:52,866 : INFO : topic diff=0.074934, rho=0.353344
2021-02-20 07:56:52,868 : DEBUG : bound: at document #

2021-02-20 07:57:06,208 : INFO : topic #1 (0.053): 0.077*"motif_60" + 0.071*"motif_184" + 0.054*"motif_218" + 0.037*"motif_166" + 0.030*"motif_98" + 0.028*"motif_284" + 0.027*"motif_171" + 0.027*"motif_94" + 0.025*"motif_181" + 0.020*"motif_158"
2021-02-20 07:57:06,209 : INFO : topic #2 (0.078): 0.042*"motif_37" + 0.038*"motif_23" + 0.036*"motif_42" + 0.031*"motif_40" + 0.030*"motif_13" + 0.027*"motif_19" + 0.023*"motif_39" + 0.018*"motif_54" + 0.017*"motif_11" + 0.017*"motif_12"
2021-02-20 07:57:06,209 : INFO : topic diff=0.069844, rho=0.333158
2021-02-20 07:57:06,211 : INFO : PROGRESS: pass 2, at document #60000/60095
2021-02-20 07:57:06,211 : DEBUG : performing inference on a chunk of 10000 documents
2021-02-20 07:57:08,381 : DEBUG : 10000/10000 documents converged within 1000 iterations
2021-02-20 07:57:08,410 : INFO : optimized alpha [0.08957468, 0.054992937, 0.081933595]
2021-02-20 07:57:08,411 : DEBUG : updating topics
2021-02-20 07:57:08,411 : INFO : merging changes from 10000 

2021-02-20 07:57:17,092 : INFO : topic diff=0.065670, rho=0.316078
2021-02-20 07:57:17,094 : INFO : PROGRESS: pass 3, at document #50000/60095
2021-02-20 07:57:17,095 : DEBUG : performing inference on a chunk of 10000 documents
2021-02-20 07:57:19,987 : DEBUG : 10000/10000 documents converged within 1000 iterations
2021-02-20 07:57:20,028 : INFO : optimized alpha [0.12883937, 0.06689222, 0.1080644]
2021-02-20 07:57:20,029 : DEBUG : updating topics
2021-02-20 07:57:20,030 : INFO : merging changes from 10000 documents into a model of 60095 documents
2021-02-20 07:57:20,031 : INFO : topic #0 (0.129): 0.034*"motif_60" + 0.024*"motif_126" + 0.023*"motif_62" + 0.023*"motif_84" + 0.018*"motif_17" + 0.016*"motif_31" + 0.016*"motif_26" + 0.015*"motif_162" + 0.014*"motif_174" + 0.014*"motif_35"
2021-02-20 07:57:20,032 : INFO : topic #1 (0.067): 0.079*"motif_60" + 0.073*"motif_184" + 0.055*"motif_218" + 0.038*"motif_166" + 0.030*"motif_98" + 0.029*"motif_284" + 0.028*"motif_171" + 0.028*"motif_94

2021-02-20 07:57:33,960 : DEBUG : updating topics
2021-02-20 07:57:33,961 : INFO : merging changes from 10000 documents into a model of 60095 documents
2021-02-20 07:57:33,962 : INFO : topic #0 (0.165): 0.033*"motif_60" + 0.024*"motif_126" + 0.024*"motif_62" + 0.022*"motif_84" + 0.017*"motif_17" + 0.016*"motif_31" + 0.015*"motif_26" + 0.015*"motif_162" + 0.014*"motif_174" + 0.014*"motif_35"
2021-02-20 07:57:33,963 : INFO : topic #1 (0.079): 0.080*"motif_60" + 0.074*"motif_184" + 0.057*"motif_218" + 0.039*"motif_166" + 0.031*"motif_98" + 0.030*"motif_284" + 0.028*"motif_171" + 0.028*"motif_94" + 0.026*"motif_181" + 0.022*"motif_219"
2021-02-20 07:57:33,964 : INFO : topic #2 (0.134): 0.044*"motif_37" + 0.040*"motif_23" + 0.038*"motif_42" + 0.032*"motif_40" + 0.031*"motif_13" + 0.028*"motif_19" + 0.024*"motif_39" + 0.018*"motif_11" + 0.018*"motif_12" + 0.017*"motif_10"
2021-02-20 07:57:33,964 : INFO : topic diff=0.056875, rho=0.301381
2021-02-20 07:57:33,967 : INFO : PROGRESS: pass 4, at 

CPU times: user 1min 47s, sys: 121 ms, total: 1min 47s
Wall time: 1min 47s


In [40]:
def likelihoodMetric(model,corpus):
    likelihood=0
    P_T=model.alpha
    for pred in lda.get_document_topics(corpus,minimum_probability=0):
        P_Xi_M=0
        for topic_no,P_Xi_T in pred:
            P_Xi_M+=P_Xi_T*P_T[topic_no]
        likelihood+=np.log10(P_Xi_M)
    print(likelihood)
    return likelihood

/home/vanka/anaconda2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [66]:
from collections import Counter

def findTopMotifs(model,corpus,data,ntop=5):
    pred_topic=[]
    for pred in lda.get_document_topics(corpus,minimum_probability=0):
        top_topic=sorted(pred,key=lambda x:-x[1])[0][0]
        pred_topic.append(top_topic)
    data['pred_topic']=pred_topic
    top_topics_data={'topic_no':[],'top_topics':[]}       
    gb=data[['motif_string','pred_topic']].groupby('pred_topic').\
    agg(lambda x: ','.join(x))
    gb['top_topics']=gb['motif_string'].\
    apply(lambda x:Counter(x.split(',')).most_common(ntop))
    print(gb['top_topics'])
    return gb

/home/vanka/anaconda2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [67]:
%%time
logging.getLogger().setLevel(logging.CRITICAL)
from gensim.models import LdaModel,LdaMulticore

eval_dict={'num_topics':[],'likelihood_metric':[]}
temp = dictionary[0]
id2word = dictionary.id2token
for num_topics in range(2,10):
    print('Num of Topic '+str(num_topics))
    lda = LdaModel(corpus, id2word=id2word, alpha='auto',chunksize=10000,
                   eta='auto',num_topics=num_topics, iterations=1000, passes = 5)
    likelihood=likelihoodMetric(lda,corpus)
    findTopMotifs(lda,corpus,data)
    eval_dict['num_topics'].append(num_topics)
    eval_dict['likelihood_metric'].append(likelihood)

/home/vanka/anaconda2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Num of Topic 2
-49363.41914238052
                                                 motif_string  \
pred_topic                                                      
0           motif_149,motif_162,motif_166,motif_166,motif_...   
1           motif_144,motif_120,motif_121,motif_128,motif_...   

                                         top_topics  
pred_topic                                           
0           [(motif_60, 46033), (motif_184, 28619)]  
1            [(motif_37, 26908), (motif_23, 24777)]  
Num of Topic 3
-49113.76904516667
                                                 motif_string  \
pred_topic                                                      
0           motif_144,motif_110,motif_121,motif_133,motif_...   
1           motif_104,motif_126,motif_126,motif_147,motif_...   
2           motif_149,motif_162,motif_166,motif_166,motif_...   

                                         top_topics  
pred_topic                                           
0            [(motif_

In [69]:
pd.DataFrame(eval_dict)

/home/vanka/anaconda2/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,num_topics,likelihood_metric
0,2,-49363.419142
1,3,-49113.769045
2,4,-50332.522068
3,5,-51430.684847
4,6,-54941.572505
5,7,-57199.418550
6,8,-51769.267887
7,9,-56834.975663
